Ch3 . 텐서플로 프로그래밍 101
======================
3.1 텐서와 그래프 실행
-------------------------------

In [5]:
import tensorflow as tf

In [6]:
hello = tf.constant('Hello, Tensorflow')
print(hello)

Tensor("Const_1:0", shape=(), dtype=string)


* hello 
> - 자료형 : tensor
> - 상수를 담고 있음

* tensor : 텐서플로에서 다양한 수학식을 계산하기 위한 가장 기본적이고 중요한 자료형
> - rank, shape 개념을 갖고 있음

* rank : 차원의 수를 나타냄
> - rank 0 : scalar
> - rank 1 : vector
> - rank 2 : matrix
> - rank 3 ~ : n-Tensor / n차원 텐서

* shape : 각 차원의 요소 갯수 -> 텐서의 구조를 설명
> - 3                               # rank 0 ; shape []
> - [1., 2. 3.]                     # rank 1 ; shape [3]
> - [[1.,2.,3.], [4., 5., 6.]]      # rank 2 ; shape [2,3]
> - [[[1., 2., 3.]], [7., 8., 9.]]] # rank 3 ; shape [2,1,3]


* dtype : 해당 텐서에 담긴 요소들의 자료형 
> string, float, int 등 가능

In [7]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


위의 결과 42가 아닌 tensor가 나오는 것을 확인할 수 있다. 

## 텐서플로 프로그램의 구조가 다음 2가지로 분리되어 있음
1. 그래프 생성
<pre>
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b)
<code>

a 에 10을 저장하고, b에 32를 저장한 다음 c에 더해 놓는 것 까지! 

2. 그래프 실행
<pre>
sess.run(c)
<code>

c라는 연산을 수행한 결과 값을 보여주게 됨!

* 그래프 : tensor들의 연산 모음
> tensorflow : 텐서와 텐서의 연산 먼저 정의 -> 그래프 만듦 -> 이후 필요할 떄 연산 실행 코드를 넣어 '원하는 지점'에 실제 연산 수행
=> 지연실행 방법 (lazy evaluation) <- 함수 프로그래밍에서 많이 사용

In [9]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a,b,c]))

sess.close()

b'Hello, Tensorflow'
[10, 32, 42]


그래프의 실행은 Session 안에서 이뤄져야 함
session 객체와 run 방법을 이용하면 된다.

In [10]:
# None은 크기가 정해지지 않았음을 의미한다.
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


X에 넣을 자료를 정의가능함
-> 텐서 모양을 (?,3)으로 정의 
-> 두번째 차원은 요소를 3개씩 가지고 있어야 한다!

In [11]:
x_data = [[1, 2, 3], [4, 5, 6]]

In [12]:
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

W와 b에 텐서플로의 변수를 생성하여 할당한다.
W는 [3,2], b는 [2,1]의 행렬형태의 텐서로, tf.random_normal 함수를 이용해 정규분포의 무작위 값으로 초기화한다.
이때, 다른 생성 함수를 사용하거나, 직접 원하는 텐서의 형태의 데이터를 만들어서 넣어줄 수도 있다.
<pre>
W = tf.Variable([[0.1, 0.1], [0.2, 0.2]. [0.3, 0.3]])
<code>

In [13]:
expr = tf.matmul(X,W) + b

입력값과 변수들을 계산할 수식을 작성한다. 
* X와 W가 행렬이기 떄문에 tf.matmul 함수를 사용한다.
* 행렬이 아닌 경우 단순히 곱셈연산자 (*) 나 tf.mul 함수를 사용한다.

### 행렬곱
* 행렬곱  A X B 에 대하여, 행렬 A의 열 수와 행렬 B의 행 수는 같아야 한다.
* 행렬곱  A X B 를 계산한 행렬 AB의 크기는 A의 행 개수와 B의 열 개수가 된다. 

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict = {X: x_data}))

sess.close()

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[-1.4167143  -0.36121407]
 [ 0.14927205  0.33650607]
 [-0.7535148   0.78867215]]
=== b ===
[[ 0.84317756]
 [-0.6308894 ]]
=== expr ===
[[ -2.5355372   3.520992 ]
 [-10.072475    4.3388176]]


tf.global_variables_initializer는 앞에서 정의한 변수들을 초기화하는 함수
> 기존에 학습한 값들을 가져와서 사용하는 것이 아닌 처음 실행하는 것이라면, 연산을 실행하기 전에 반드시 이 함수를 이용해 변수들을 초기화 해야한다.

feed_dict 매개변수는 그래프를 실행할 때 사용할 입력값을 지정한다.

expr 수식에는 X, W, b의 값을 사용했는데, 이중 X가 플레이스홀더라 X에 값을 넣어주지 않는다면 계산에 사용할 값이 없으므로 에라가 난다. 
따라서 미리 정의해둔 x_data를 X의 값으로 사용한다. 

In [18]:
##전체 코드
import tensorflow as tf

X = tf.placeholder(tf.float32, [None, 3])
print(X)

x_data = [[1,2,3], [4,5,6]]

W = tf.Variable(tf.random_normal([3,2]))
b = tf.Variable(tf.random_normal([2,1]))

expr = tf.matmul(X, W) + b

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict = {X : x_data}))

sess.close()

Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[-1.276627   -0.2978756 ]
 [ 0.11252449 -0.6578365 ]
 [ 2.2665966   1.675789  ]]
=== b ===
[[ 0.7384755 ]
 [-0.16631424]]
=== expr ===
[[6.4866867 4.1522937]
 [8.8893795 5.4077353]]
